In [ ]:
%matplotlib inline
import fnmatch
import os
import pandas as pd
from tqdm import tqdm

pd.set_option("display.max_columns",40)

def parse_fname(fname, df):
    fname = os.path.basename(fname)
    fn = os.path.splitext(fname)[0]
    d = dict(zip(('name', 'motive', 'col', 'type'), fn.split('-')))
    for key, val in d.items():
        df[key] = val.strip()
    df['file'] = fname

In [ ]:
matches = []
for root, dirnames, filenames in os.walk('KPI-TXT'):
    for filename in fnmatch.filter(filenames, '*.txt'):
        matches.append(os.path.join(root, filename))

In [ ]:
df = pd.DataFrame()
for f in tqdm(matches):
    data = pd.read_csv(f, sep='\t')
    parse_fname(f,data)
    if "White Space" in data['Area of Interest'].values:
        wsp = data[(data['Area of Interest'] == "White Space")]
        out = data[(data['Area of Interest'] == "ausserhalb")]
        wsp_i, wsp_af, wsp_fc = wsp.index, wsp['Average fixation [ms]'].values[0], wsp['Fixation count'].values[0]
        out_i, out_af, out_fc = out.index, out['Average fixation [ms]'].values[0], out['Fixation count'].values[0]
        data.set_value(out_i, 'Average fixation [ms]', (wsp_af*wsp_fc+out_af*out_fc)/(wsp_fc+out_fc))
        data.set_value(out_i, 'Fixation count', wsp_fc+out_fc)
        data.drop(wsp_i,inplace=True)
    df = df.append(data, ignore_index=True)

In [ ]:
df['Total Fixation [ms]'] = df['Fixation count'] * df['Average fixation [ms]']
df.drop(['Unnamed: 15'], axis=1, inplace=True)

In [ ]:
df1 = df[['col','motive','type','Area of Interest', 'Fixation count', 'Total Fixation [ms]']]
group = df1.groupby(['motive','col','Area of Interest']).sum()
res = group.apply(lambda x: x.sort_values())
res

In [ ]:
#write things to excel
writer = pd.ExcelWriter('output.xlsx')
group.to_excel(writer,'Groups')
df.to_excel(writer,'Data')
writer.save()